# Automatic Execution

Configuring the manager_llm parameter is crucial for the hierarchical process.

 The system requires a manager LLM to be set up for proper function, ensuring tailored decision-making.

## Using a Custom Manager Agent

Alternatively, you can create a custom manager agent with specific attributes tailored to your project’s management needs.
 This gives you more control over the manager’s behavior and capabilities.

In [5]:
from crewai import Agent, Task, Crew, Process,LLM
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.2
)

In [ ]:
# =========================
# AGENTS DEFINITION
# =========================

# 1. Market Researcher Agent
market_researcher = Agent(
    role="Market Research Specialist",
    goal="Research target audience, competitors, and trending topics in the industry",
    backstory="""You are an expert market researcher with 10 years of experience.
    You know how to find valuable insights about audiences, trends, and competitors.
    You provide detailed, data-driven research reports.""",
    llm=llm,
    verbose=True
)

# 2. SEO Specialist Agent
seo_specialist = Agent(
    role="SEO Expert",
    goal="Identify high-value keywords and SEO optimization strategies",
    backstory="""You are a seasoned SEO professional who understands search algorithms.
    You find the perfect keywords that balance search volume with ranking difficulty.
    You provide actionable SEO recommendations.""",
    llm=llm,
    verbose=True
)

# 3. Content Writer Agent
content_writer = Agent(
    role="Senior Content Writer",
    goal="Create engaging, well-structured blog posts that resonate with the audience",
    backstory="""You are a talented writer with expertise in creating compelling content.
    You know how to weave research and keywords into natural, engaging narratives.
    Your content drives engagement and conversions.""",
    llm=llm,
    verbose=True
)

# 4. Quality Editor Agent
quality_editor = Agent(
    role="Editorial Quality Checker",
    goal="Review content for accuracy, completeness, and quality standards",
    backstory="""You are a meticulous editor with a keen eye for detail.
    You ensure every piece meets high standards for accuracy, clarity, and completeness.
    You're not afraid to send work back if it needs improvement.""",
    llm=llm,
    verbose=True
)

# =========================
# TASKS DEFINITION
# =========================

# Task 1: Market Research
research_task = Task(
    description="""Research the topic: {topic}
    
    Provide:
    - Target audience demographics and pain points
    - Top 3 competitor articles on this topic
    - Current trends related to this topic
    - Key questions the audience is asking
    
    Make sure your research is comprehensive and recent.""",
    agent=market_researcher,
    expected_output="Detailed market research report with audience insights, competitor analysis, and trends"
)

# Task 2: SEO Keyword Research
seo_task = Task(
    description="""Based on the market research, identify SEO opportunities for: {topic}
    
    Provide:
    - 5-10 primary and secondary keywords
    - Search intent for each keyword
    - Keyword difficulty and search volume estimates
    - Recommended content structure for SEO
    
    Ensure keywords are relevant and achievable.""",
    agent=seo_specialist,
    expected_output="SEO keyword strategy with search volumes, difficulty scores, and content recommendations"
)

# Task 3: Content Writing
writing_task = Task(
    description="""Write a comprehensive blog post about: {topic}
    
    Requirements:
    - Use the market research insights to address audience pain points
    - Naturally incorporate the SEO keywords identified
    - Create an engaging introduction and strong conclusion
    - Include practical examples and actionable tips
    - Aim for 1500-2000 words
    - Use clear headings and subheadings
    
    The content must be original, valuable, and well-structured.""",
    agent=content_writer,
    expected_output="Complete blog post draft with proper structure, SEO optimization, and engaging content"
)

# Task 4: Quality Check
quality_task = Task(
    description="""Review the blog post for quality and completeness.
    
    Check:
    - Does it address the audience pain points from research?
    - Are SEO keywords naturally incorporated?
    - Is the content comprehensive (1500-2000 words)?
    - Is it well-structured with clear headings?
    - Are there practical examples and actionable tips?
    - Is the tone engaging and professional?
    
    If ANY requirement is not met, flag it clearly and request improvements.
    If everything looks good, approve the content.""",
    agent=quality_editor,
    expected_output="Quality assessment report with approval or specific improvement requests"
)

# =========================
# CREW WITH HIERARCHICAL PROCESS
# =========================

# # Option 1: Let CrewAI automatically create a manager
# content_crew = Crew(
#     agents=[market_researcher, seo_specialist, content_writer, quality_editor],
#     tasks=[research_task, seo_task, writing_task, quality_task],
#     process=Process.hierarchical,  # This enables hierarchical mode
#     verbose=True,
#     manager_llm = "try different openai model "
#     memory=True  # Enables crew memory for better context
# )

# Option 2: Explicitly define your own manager (uncomment to use)
manager_agent = Agent(
    role="Content Marketing Manager",
    goal="Coordinate the content creation process and ensure high-quality output",
    backstory='''You are an experienced content marketing manager who knows how to 
    orchestrate teams effectively. You delegate tasks wisely, validate outputs, 
    and ensure the final content meets all requirements. You're not afraid to 
    request revisions when needed.''',
    llm=llm,
    allow_delegation=True
)

content_crew = Crew(
    agents=[market_researcher, seo_specialist, content_writer, quality_editor],
    tasks=[research_task, seo_task, writing_task, quality_task],
    process=Process.hierarchical,
    manager_agent=manager_agent,  # Use your custom manager
    verbose=True,
)

# =========================
# EXECUTION
# =========================

if __name__ == "__main__":
    # Input topic
    inputs = {
        "topic": "How AI is Transforming Small Business Marketing in 2025"
    }
    
    print("\n🚀 Starting Content Marketing Crew (Hierarchical Process)...\n")
    print("=" * 70)
    
    # The manager will:
    # 1. Assign research to Market Researcher
    # 2. Validate research output
    # 3. Assign SEO task to SEO Specialist
    # 4. Validate SEO output
    # 5. Assign writing to Content Writer
    # 6. Assign quality check to Quality Editor
    # 7. If quality check fails -> Go back to research/writing as needed
    # 8. If quality check passes -> Complete the workflow
    
    result = content_crew.kickoff(inputs=inputs)
    
    print("\n" + "=" * 70)
    print("\n✅ FINAL OUTPUT:\n")
    print(result)


🚀 Starting Content Marketing Crew (Hierarchical Process)...



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  cc0acd6a-f7a5-4c83-bc46-77a3cdcdec25                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Research the topic: How AI is Transforming Small Business Marketing in 2025                              │
│                                                                                                                 │
│      Provide:                                                                                                   │
│      - Target audience demographics and pain points                                                             │
│      - Top 3 competitor articles on this topic                                                                  │
│      - Current trends related to this topic                                                                     │
│      - Key questions the audience is asking                                                                     │
│                                                                                                                 │
│      Make sure your research is comprehensive and recent.                                                       │
│  ID: 3997c8ea-dfa3-4826-9e03-b406a72058f0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 🧠 Memory Retrieval ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Memory Retrieval Started                                                                                       │
│  Status: Retrieving...                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:Error during short_term search: The OPENAI_API_KEY environment variable is not set.
Traceback (most recent call last):
  File "d:\CREWAI_CRASH_COURSE\crewai-course\Lib\site-packages\crewai\memory\storage\rag_storage.py", line 240, in search
    client = self._get_client()
             ^^^^^^^^^^^^^^^^^^
  File "d:\CREWAI_CRASH_COURSE\crewai-course\Lib\site-packages\crewai\memory\storage\rag_storage.py", line 108, in _get_client
    return self._client if self._client else get_rag_client()
                                             ^^^^^^^^^^^^^^^^
  File "d:\CREWAI_CRASH_COURSE\crewai-course\Lib\site-packages\crewai\rag\config\utils.py", line 70, in get_rag_client
    get_rag_config()
  File "d:\CREWAI_CRASH_COURSE\crewai-course\Lib\site-packages\crewai\rag\config\utils.py", line 50, in get_rag_config
    default_config = config_class()
                     ^^^^^^^^^^^^^^
  File "d:\CREWAI_CRASH_COURSE\crewai-course\Lib\site-packages\pydantic\_internal\_dataclasses.py", li

...

Would you like to view your execution traces? [y/N] (20s timeout): 

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jc8r55vqeq6v4yza1hzy47vz` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Used 10386, Requested 3953. Please try again in 11.694999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}




┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                                             │
│  Info: Tracing has been disabled.                                           │
│                                                                             │
│  Your preference has been saved. Future Crew/Flow executions will not       │
│  collect traces.                                                            │
│                                                                             │
│  To enable tracing later, do any one of these:                              │
│  • Set tracing=True in your Crew/Flow code                                  │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file              │
│  • Run: crewai traces enable                                                │
│                                                                             │
└─────────────────────────────────────

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  cc0acd6a-f7a5-4c83-bc46-77a3cdcdec25                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯